In [1]:
import sys, os
sys.path.insert(0, os.path.abspath("../"))
#from controlunit import channels as ch
#from controlunit import conversions
import yaml

In [2]:
def load_settings(path_to_file):
    with open(path_to_file, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [3]:
st = load_settings("settings.yml")

In [4]:
st

{'Data Folder': '~/work/cudata',
 'Path is Local': True,
 'Sampling Rate': 0.1,
 'Max Temperature': 1000,
 'MAX6675 Channel': 0,
 'Heater GPIO': 17,
 'LED GPIO': 27,
 'Temperature Columns': ['date', 'time', 'T', 'PresetT'],
 'ADC Channels': {'Pd': {'Channel': 0,
   'Gain': 10,
   'Description': 'Ionization Gauge (downstream)',
   'Conversion Function': 'Ionization Gauge'},
  'Pu': {'Channel': 1,
   'Gain': 10,
   'Description': 'Pfeffer  Single Gauge (upstream)',
   'Conversion Function': 'Pfeiffer Single Gauge'},
  'Ip': {'Channel': 2,
   'Gain': 5,
   'Description': 'Plasma Current',
   'Conversion Function': 'Hall Sensor'},
  'Bu': {'Channel': 3,
   'Gain': 1,
   'Description': 'Baratron MKS 627 FS = 1 Torr (upstream)',
   'Conversion Function': 'Baratron',
   'Full Scale': 1.0},
  'Bd': {'Channel': 4,
   'Gain': 1,
   'Description': 'Baratron MKS 628B.1TDF2b FS = 0.1 Torr (downstream)',
   'Conversion Function': 'Baratron',
   'Full Scale': 0.1}},
 'ADC Additional Columns': ['date'

In [5]:
st['ADC Channels']['Bd']['Full Scale']

0.1

In [6]:
ad1 = list(st['ADC Channels'])[0]

In [7]:
class AdcChannelProps:
    """
    ADC channel properties
    """

    def __init__(self, *arg, **kws) -> None:
        """
        Arguments
        ----------
        [name,channel, gain, description, conversion]
        """
        self.name = arg[0]        
        self.channel = kws['Channel']
        self.gain = kws['Gain']        
        self.description = kws['Description']
        self.conversion_id = kws['Conversion Function']
        self.full_scale = kws.get('Full Scale',None)
        self.set_conversion_function()

    def set_conversion_function(self):
        """
        Select conversion function from a dict by conversion_id
        """
        from controlunit.conversions import ionization_gauge, hall_current_sensor, pfeiffer_single_gauge, baratron
        
        conversions = {
            "Ionization Gauge": ionization_gauge,
            "Pfeiffer Single Gauge": pfeiffer_single_gauge,
            "Hall Sensor": hall_current_sensor,
            "No Conversion": lambda v: v
        }

        if self.conversion_id == 'Baratron':
            self.conversion = lambda v : baratron(v,self.full_scale)
            return
        
        self.conversion = conversions[self.conversion_id]


In [8]:
channels = {name:AdcChannelProps(name,**st['ADC Channels'][name]) for name in list(st['ADC Channels'])}

In [21]:
channels['Ip'].conversion(3)

2.4

In [11]:
channels['Pd'].conversion_id

'Ionization Gauge'